# 分析周内不同时间的涨跌幅与上涨概率
由于使用两个账户进行定投，定投周期为每周，一个在周一进行定投，另一个在周四进行定投，昨天定投扣款后，总感觉周一定投时上涨的概率比周四上涨的概率大，于是有了本篇分析，分析的结果如下：
1. 从历史角度来看（2007年至今），有且仅有周四的涨跌幅为负数，且有且仅有周四上涨概率小于50%
2. 统计2018年年初至今的数据，周一依旧比周四涨幅与上涨概率大
3. 2018年至今，周三的行情比周四还差，无论是涨跌幅还是概率都差不少

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import  jqdatasdk
import talib

In [3]:
jqdatasdk.auth(tel, pwd)

auth success


# 选取历史（2007年1月1日到2018年11月19日）数据进行计算

In [121]:
hs300_df = jqdatasdk.get_price('000300.XSHG', start_date='2007-01-01', end_date='2018-11-30', frequency='daily', fields=['open', 'close', 'high', 'low']) 

In [122]:
hs300_df.head()

,open,close,high,low
2007-01-04,2073.250,2067.092,2139.494,2054.242
2007-01-05,2051.152,2072.883,2083.402,2030.762
2007-01-08,2072.005,2131.556,2131.737,2071.721
2007-01-09,2137.485,2200.086,2201.360,2128.064
2007-01-10,2210.757,2255.966,2255.966,2194.770


In [123]:
hs300_df['date'] = hs300_df.index

In [124]:
import time,datetime

# 计算当天为周几
def get_week_day(date):
  week_day_dict = {
    0 : '星期一',
    1 : '星期二',
    2 : '星期三',
    3 : '星期四',
    4 : '星期五',
    5 : '星期六',
    6 : '星期天',
  }
  day = date.weekday()
  return week_day_dict[day]

In [125]:
hs300_df['weekday'] = hs300_df['date'].apply(lambda x : get_week_day(x))

In [126]:
hs300_df.head()

,open,close,high,low,date,weekday
2007-01-04,2073.250,2067.092,2139.494,2054.242,2007-01-04,星期四
2007-01-05,2051.152,2072.883,2083.402,2030.762,2007-01-05,星期五
2007-01-08,2072.005,2131.556,2131.737,2071.721,2007-01-08,星期一
2007-01-09,2137.485,2200.086,2201.360,2128.064,2007-01-09,星期二
2007-01-10,2210.757,2255.966,2255.966,2194.770,2007-01-10,星期三


In [127]:
# 计算当日的收益率
hs300_df['dr%'] = (hs300_df['close'] - hs300_df['open']) / hs300_df['open'] * 100

In [128]:
hs300_df.head()

,open,close,high,low,date,weekday,dr%
2007-01-04,2073.250,2067.092,2139.494,2054.242,2007-01-04,星期四,-0.297022
2007-01-05,2051.152,2072.883,2083.402,2030.762,2007-01-05,星期五,1.059453
2007-01-08,2072.005,2131.556,2131.737,2071.721,2007-01-08,星期一,2.874076
2007-01-09,2137.485,2200.086,2201.360,2128.064,2007-01-09,星期二,2.928722
2007-01-10,2210.757,2255.966,2255.966,2194.770,2007-01-10,星期三,2.044956


In [129]:
weekday_group = hs300_df.groupby('weekday')

In [130]:
# 计算周一到周五的每日平均涨跌幅
weekday_group['dr%'].mean()

weekday
星期一    0.151212
星期三    0.196211
星期二    0.105253
星期五    0.191623
星期四   -0.045323
Name: dr%, dtype: float64

In [131]:
# 计算周一到周五的每日的上涨概率
weekday_group['dr%'].agg(lambda x : len(x[x > 0]) / len(x))

weekday
星期一    0.570922
星期三    0.524618
星期二    0.566781
星期五    0.561313
星期四    0.469983
Name: dr%, dtype: float64

# 选取今年（2018年1月1日到2018年11月30日）的数据进行计算

In [132]:
hs300_df = jqdatasdk.get_price('000300.XSHG', start_date='2018-01-01', end_date='2018-11-30', frequency='daily', fields=['open', 'close', 'high', 'low']) 

In [133]:
hs300_df['date'] = hs300_df.index
hs300_df['weekday'] = hs300_df['date'].apply(lambda x : get_week_day(x))
hs300_df['dr%'] = (hs300_df['close'] - hs300_df['open']) / hs300_df['open'] * 100
weekday_group = hs300_df.groupby('weekday')

In [134]:
# 计算周一到周五的每日平均涨跌幅
weekday_group['dr%'].mean()

weekday
星期一    0.105922
星期三   -0.335314
星期二    0.227191
星期五    0.161232
星期四   -0.060633
Name: dr%, dtype: float64

In [135]:
# 计算周一到周五的每日的上涨概率
weekday_group['dr%'].agg(lambda x : len(x[x > 0]) / len(x))

weekday
星期一    0.547619
星期三    0.347826
星期二    0.555556
星期五    0.600000
星期四    0.400000
Name: dr%, dtype: float64